##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Usar DTensors con Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/dtensor_keras_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a> </td>
</table>

## Visión general

En este tutorial aprenderá a usar DTensor con Keras.

Gracias a la integración de DTensor con Keras, puede reutilizar sus capas y modelos existentes de Keras para construir y entrenar modelos distribuidos de aprendizaje automático.

Entrenará un modelo de clasificación multicapa con los datos MNIST. Se demostrará cómo configurar el modelo de subclasificación, el modelo secuencial y el modelo funcional.

Este tutorial asume que ya ha leído la [Guía de programación del DTensor](/guide/dtensor_overview), y que está familiarizado con conceptos básicos del DTensor como `Mesh` y `Layout`.

Este tutorial se basa en https://www.tensorflow.org/datasets/keras_example.

## Preparar

DTensor forma parte de la versión 2.9.0 de TensorFlow.

In [ ]:
!pip install --quiet --upgrade --pre tensorflow tensorflow-datasets

A continuación, importe `tensorflow` y `tensorflow.experimental.dtensor`, y configure TensorFlow para que use 8 CPU virtuales.

Aunque este ejemplo usa CPUs, DTensor funciona igual en dispositivos con CPU, GPU o TPU.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.experimental import dtensor

In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(
        phy_devices[0], 
        [tf.config.LogicalDeviceConfiguration()] * ncpu)
  
configure_virtual_cpus(8)
tf.config.list_logical_devices('CPU')

devices = [f'CPU:{i}' for i in range(8)]

## Generadores de números pseudoaleatorios deterministas

Algo que debe tener en cuenta es que la API DTensor requiere que cada uno de los clientes en ejecución tenga las mismas semillas aleatorias, para así poder tener un comportamiento determinista a la hora de inicializar las ponderaciones. Puede conseguirlo si establece las semillas globales en keras mediante `tf.keras.utils.set_random_seed()`.

In [ ]:
tf.keras.backend.experimental.enable_tf_random_generator()
tf.keras.utils.set_random_seed(1337)

## Crear una malla Paralela de Datos

Este tutorial demuestra el entrenamiento Paralelo de Datos. Adaptarse al entrenamiento Paralelo de Modelos y Paralelo Espacial puede ser tan sencillo como cambiar a un conjunto diferente de objetos `Layout`. Consulte el [Tutorial detallado de ML con DTensor](https://www.tensorflow.org/tutorials/distribute/dtensor_ml_tutorial) para saber más sobre el entrenamiento distribuido más allá de Paralelo de Datos.

El entrenamiento Paralelo de Datos es un esquema de entrenamiento paralelo muy usado, que también usa, por ejemplo, `tf.distribute.MirroredStrategy`.

Con DTensor, un bucle de entrenamiento Paralelo de Datos usa un `Mesh` que consiste en una única dimensión "lote", donde cada dispositivo ejecuta una réplica del modelo que recibe un fragmento del lote global.


In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=devices)

Como cada dispositivo ejecuta una réplica completa del modelo, las variables del modelo estarán totalmente replicadas en toda la malla (sin fragmentar). Por ejemplo, un Layout totalmente replicada para una ponderación de rango 2 en esta `Mesh` sería la siguiente:

In [ ]:
example_weight_layout = dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)  # or
example_weight_layout = dtensor.Layout.replicated(mesh, rank=2)

Una disposición para un tensor de datos de rango 2 en esta `Mesh` sería fragmentada a lo largo de la primera dimensión (a veces conocida como `batch_sharded`),

In [ ]:
example_data_layout = dtensor.Layout(['batch', dtensor.UNSHARDED], mesh)  # or
example_data_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)

## Crear capas Keras con disposición

En el esquema paralelo de datos, normalmente usted crea las ponderaciones de su modelo con una disposición totalmente replicada, de modo que cada réplica del modelo pueda hacer cálculos con los datos de entrada fragmentados.

Para configurar la información de disposición de las ponderaciones de sus capas, Keras ha expuesto un parámetro extra en el constructor de capa para la mayoría de las capas incorporadas.

El siguiente ejemplo construye un pequeño modelo de clasificación de imágenes con una distribución de ponderaciones totalmente replicada. Puede especificar la información de layout `kernel` y `bias` en `tf.keras.layers.Dense` mediante los argumentos `kernel_layout` y `bias_layout`. La mayoría de las capas incorporadas de keras están preparadas para especificar explícitamente la `Layout` para las ponderaciones de la capa.

In [ ]:
unsharded_layout_2d = dtensor.Layout.replicated(mesh, 2)
unsharded_layout_1d = dtensor.Layout.replicated(mesh, 1)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, 
                        activation='relu',
                        name='d1',
                        kernel_layout=unsharded_layout_2d, 
                        bias_layout=unsharded_layout_1d),
  tf.keras.layers.Dense(10,
                        name='d2',
                        kernel_layout=unsharded_layout_2d, 
                        bias_layout=unsharded_layout_1d)
])

Puede comprobar la información sobre layout examinando la propiedad `layout` de las ponderaciones.

In [ ]:
for weight in model.weights:
  print(f'Weight name: {weight.name} with layout: {weight.layout}')
  break

## Cargar un conjunto de datos y crear una canalización de entrada

Cargue un conjunto de datos MNIST y configure algunas canalizaciones de entrada de preprocesamiento para él. El conjunto de datos en sí no está asociado a ninguna información de disposición DTensor. Hay planes para mejorar la integración del DTensor Keras con `tf.data` en futuras versiones de TensorFlow.


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [ ]:
batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Definir la lógica de entrenamiento del modelo

A continuación, defina la lógica de entrenamiento y evaluación del modelo.

A partir de TensorFlow 2.9, debe escribir un bucle de entrenamiento personalizado para un modelo Keras habilitado para DTensor. Esto sirve para empaquetar los datos de entrada con la información de disposición adecuada, la cual no está integrada con las funciones estándar `tf.keras.Model.fit()` o `tf.keras.Model.eval()` de Keras. En las próximas versiones habrá más compatibilidad con `tf.data`. 

In [ ]:
@tf.function
def train_step(model, x, y, optimizer, metrics):
  with tf.GradientTape() as tape:
    logits = model(x, training=True)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))
    
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'loss': loss_per_sample}
  return results

In [ ]:
@tf.function
def eval_step(model, x, y, metrics):
  logits = model(x, training=False)
  loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'eval_loss': loss_per_sample}
  return results

In [ ]:
def pack_dtensor_inputs(images, labels, image_layout, label_layout):
  num_local_devices = image_layout.mesh.num_local_devices()
  images = tf.split(images, num_local_devices)
  labels = tf.split(labels, num_local_devices)
  images = dtensor.pack(images, image_layout)
  labels = dtensor.pack(labels, label_layout)
  return  images, labels

## Métricas y optimizadores

Cuando use la API DTensor con `Metric` y `Optimizer` de Keras, tendrá que dar la información adicional de Mesh, para que las variables de estado internas y los tensores puedan funcionar con las variables del modelo.

- Para un optimizador, DTensor introduce un nuevo namespace experimental `keras.dtensor.experimental.optimizers`, donde muchos optimizadores Keras existentes se amplían para recibir un argumento adicional `mesh`. En futuras versiones, es posible que se fusione con los optimizadores centrales de Keras.

- En cuanto a las métricas, puede especificar directamente la `mesh` al constructor como argumento para convertirlo en una `Metric` compatible con DTensor.

In [ ]:
optimizer = tf.keras.dtensor.experimental.optimizers.Adam(0.01, mesh=mesh)
metrics = {'accuracy': tf.keras.metrics.SparseCategoricalAccuracy(mesh=mesh)}
eval_metrics = {'eval_accuracy': tf.keras.metrics.SparseCategoricalAccuracy(mesh=mesh)}

## Entrenar el modelo

El siguiente ejemplo fragmenta los datos de la canalización de entrada en la dimensión de lote, y entrena con el modelo, que tiene ponderaciones totalmente replicadas.

Con 3 épocas, el modelo debería alcanzar aproximadamente un 97 % de precisión.

In [ ]:
num_epochs = 3

image_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=4)
label_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

for epoch in range(num_epochs):
  print("============================") 
  print("Epoch: ", epoch)
  for metric in metrics.values():
    metric.reset_state()
  step = 0
  results = {}
  pbar = tf.keras.utils.Progbar(target=None, stateful_metrics=[])
  for input in ds_train:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)

    results.update(train_step(model, images, labels, optimizer, metrics))
    for metric_name, metric in metrics.items():
      results[metric_name] = metric.result()

    pbar.update(step, values=results.items(), finalize=False)
    step += 1
  pbar.update(step, values=results.items(), finalize=True)

  for metric in eval_metrics.values():
    metric.reset_state()
  for input in ds_test:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)
    results.update(eval_step(model, images, labels, eval_metrics))

  for metric_name, metric in eval_metrics.items():
    results[metric_name] = metric.result()
  
  for metric_name, metric in results.items():
    print(f"{metric_name}: {metric.numpy()}")


## Especificar Layout para código de modelo existente

Muchas veces usted tiene modelos que funcionan bien para su caso de uso. Especificar la información `Layout` a cada capa individual dentro del modelo supondrá una gran cantidad de trabajo que requerirá muchas ediciones.

Si quiere convertir fácilmente su modelo Keras existente para que funcione con la API DTensor, puede usar la nueva API `dtensor.LayoutMap` que le permite especificar la `Layout` desde un punto de vista global.

Lo primero será crear una instancia `LayoutMap`, que es un objeto similar a un diccionario que contiene todos los `Layout` que quiera especificar para las ponderaciones de tu modelo.

`LayoutMap` necesita una instancia `Mesh` en el init, que puede usarse para crear `Layout` replicados por defecto para cualquier ponderación que no tenga Layout configurado. Si quiere que todas las ponderaciones de su modelo estén totalmente replicadas, puede indicar un `LayoutMap` vacío, y la malla predeterminada se usará para crear un `Layout` replicado.

`LayoutMap` usa una cadena como clave y un `Layout` como valor. Hay una diferencia de comportamiento entre un diccionario Python normal y esta clase. La cadena clave se tratará como una expresión regular al recuperar el valor

### Modelo subclaseados

Considere el siguiente modelo definido usando la sintaxis Model de subclase de Keras.

In [ ]:
class SubclassedModel(tf.keras.Model):

  def __init__(self, name=None):
    super().__init__(name=name)
    self.feature = tf.keras.layers.Dense(16)
    self.feature_2 = tf.keras.layers.Dense(24)
    self.dropout = tf.keras.layers.Dropout(0.1)

  def call(self, inputs, training=None):
    x = self.feature(inputs)
    x = self.dropout(x, training=training)
    return self.feature_2(x)

Hay 4 ponderaciones en este modelo, que son `kernel` y `bias` para dos capas `Dense`. Cada una de ellas se mapea en función de la ruta del objeto:

- `model.feature.kernel`
- `model.feature.bias`
- `model.feature_2.kernel`
- `model.feature_2.bias`

Nota: Para los Modelos subclaseados, se usa el nombre del atributo, en lugar del atributo `.name` de la capa, como clave para recuperar el Esquema del mapeado. Esto es consistente con la convención seguida por la aplicación de puntos de verificación de `tf.Module`. Para modelos complejos con más de unas pocas capas, puede [inspeccionar manualmente los puntos de verificación](https://www.tensorflow.org/guide/checkpoint#manually_inspecting_checkpoints) para ver los mapeos de atributos.

Ahora defina el siguiente `LayoutMap` y aplíquelo al modelo.

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

with layout_map.scope():
  subclassed_model = SubclassedModel()

Las ponderaciones del modelo se crean en la primera llamada, así que llame al modelo con una entrada DTensor y confirme que las ponderaciones tienen las distribuciones esperadas.

In [ ]:
dtensor_input = dtensor.copy_to_mesh(tf.zeros((16, 16)), layout=unsharded_layout_2d)
# Trigger the weights creation for subclass model
subclassed_model(dtensor_input)

print(subclassed_model.feature.kernel.layout)

Así, puede mapear rápidamente el `Layout` a sus modelos sin actualizar el código existente. 

### Modelos Secuencial y Funcional

Para los modelos funcional y secuencial de keras, también puede usar `LayoutMap`.

Nota: Para los modelos funcional y secuencial, los mapeos son ligeramente distintos. Las capas del modelo no tienen un atributo público asociado al modelo (aunque puede acceder a ellas a través de `model.layers` como una lista). Use el nombre de la cadena como clave en este caso. Se garantiza que el nombre de la cadena es único dentro de un modelo.

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

In [ ]:
with layout_map.scope():
  inputs = tf.keras.Input((16,), batch_size=16)
  x = tf.keras.layers.Dense(16, name='feature')(inputs)
  x = tf.keras.layers.Dropout(0.1)(x)
  output = tf.keras.layers.Dense(32, name='feature_2')(x)
  model = tf.keras.Model(inputs, output)

print(model.layers[1].kernel.layout)

In [ ]:
with layout_map.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(16, name='feature', input_shape=(16,)),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(32, name='feature_2')
  ])

print(model.layers[2].kernel.layout)